# Gale Shapley Matching with Bandits

To decide a matching between $\mathcal{B}$ and $\mathcal{L}$, we introduce the binary decision variable $\mathbf{x}$ := $(x_{bl})_{(b, l)\in \mathcal{B} \times \mathcal{L}}$ such that $x_{bl}$ = 1 if the loan from lender $l$ is assigned and accepted by borrower $b$ and 0 otherwise. It is supposed that when the borrower $b$ and the lender $l$ are matched, the borrower and the lender gain utilities of $u_b(h)$ and $u_l(h)$ respectively. To simplify  our settings, we assume that all utilities $u_b(h)$ and $u_l(h)$ are non-negative, and that for a lender $l$, $u_b(l)$ $\neq$ $u_{b'}(l)$, for $b \neq b'$ and similarly, for a borrower $b$, $u_l(b)$ $\neq$ $u_{l'}(b)$, for $l \neq l'$. The borrower-lender pair $(b, l)$ yields the utility of $u_{bl}:= u_b + u_l $ and the total utility of a matching $\mathbf{x} \in {0, 1}^{|\mathcal{B}| \times |\mathcal{L}|}$ is given by

\begin{equation*}
    \sum_{b \in \mathcal{B}} \sum_{l \in \mathcal{L}} u_{bl} x_{bl}
\end{equation*}
In our work, we consider a \textcolor{red}{many-many matching} where each borrower is matched to many lenders and each lender is also matched to multiple borrowers. For a matching $\mathbf{x}$, let $m_x(b)\in \mathcal{L}$ be the lender to which the borrower $b$ is assigned, and $M_x(l) \subseteq \mathcal{B}$ be the set of borrowers that are assigned to lender $l$, that is,
\begin{align*}
    m_x(b) = l \Longleftrightarrow x_{bl} = 1 \\
    M_x(l) := \{l \in \mathcal{L} \ | \ x_{bl} = 1\}
\end{align*}
\\

\noindent \textbf{Definition 1} Let $\mathbf{x} \in \{0, 1\}^{|\mathcal{B}| \times |\mathcal{L}|}$ be a matching. A pair $(b, l) \in \mathcal{B} \times \mathcal{L}$ is a blocking pair for $\mathbf{x}$ if the following conditions are satisfied:

\begin{enumerate}
    \item $x_{bl}=0$
    \item $x(b)$ is null or $u_b(l)$ $>$ $u_{b}(m_x(b))$
    \item $|M_x(b)|$ $< q_b$ or $\exists$ $l' \in M_x(b)$ and such that $u_b(l) > u_b(l')$.
\end{enumerate} 
\vspace*{.1in}

Since the individual rationality condition is fulfilled from Assumption 1 for all $(b, l) \in \mathcal{B} \times \mathcal{L}$, a stable matching is defined based on the absence of blocking pairs as follows: a matching $\mathbf{x}$ is stable if it does not have any blocking pairs. \\

\noindent \textbf{Definition 2.} A matching $\mathbf{x}$ is stable if it does not have blocking pairs. The stable matching problem can be characterized by the following linear inequalities as mentioned in \cite{baiou2000stable} and is formalized using the following inequality:

\begin{equation} \label{eq:eq1}
    q_b x_{bl} + q_b\sum_{l' \succ_b l }x_{bl'} + \sum_{b' \succ_{l} b} x_{b'l} \geq q_l
\end{equation}
The proof of the statement can be found in the same paper \cite{baiou2000stable}. We use this notation to arrive at a linear program that maximizes the utility of the matching while minimizing the number of blocking pairs. Defining a binary decision variable $\mathbf{w}$ := $(w_{b, l})$ $\in \{0, 1\}^{|\mathcal{B}| \times |\mathcal{L}|}$, we use the following constraint: 

\begin{equation}
        q_b x_{bl} + q_b\sum_{l' \succ_b l }x_{bl'} + \sum_{b' \succ_{l} b} x_{b'l} \geq q_l (1- w_{bl})
\end{equation}

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
import random


c = 9                     # knapsack capacity
p = [6, 5, 8, 9, 6, 7, 3] # item profits
w = [2, 3, 6, 7, 5, 9, 4] # item weights

In [24]:
def model_gs_matching(n_b, n_l, u_b, u_l, c, q, lambda_1, lamda_2, LogToConsole=True, TimeLimit=60):
    model = Model()
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit = TimeLimit # seconds
    x = {}
    for b_idx in range(1, n_b+1):
        x[b_idx] = {}
        w[b_idx] = {}
        for l_idx in range(1, n_l+1): 
            x_name = "x_{}_{}".format(b_idx, l_idx)
            x[b_idx][l_idx] = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY, name=x_name)
            w_name = "w_{}_{}".format(b_idx, l_idx)
            w[b_idx][l_idx] = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY, name=w_name)

    for l_idx in range(1, n_l+1):
        model.addConstr(quicksum(x[b_idx][l_idx] for b_idx in range(1, n_b+1)) <= 1)
    
    for b_idx in range(1, n_b+1):
        model.addConstr(quicksum((q[l_idx]*x[b_idx][l_idx]) for l_idx in range(1, n_l+1)) >= c[b_idx])
    
    for b_ix in range(1, n_b+1):
        for l_idx in range(1, n_l+1):
            constr_obj_1 = c[b_idx]*x[b_idx][l_idx]
            constr_obj_2 = 0
            constr_obj_3 = 0
            
            for b_idx_2 in range(1, n_b+1):
                if b_idx != b_idx_2:
                    if u_l[l_idx][b_idx-1] < u_l[l_idx][b_idx_2-1]:
                        constr_obj_2 += (x[b_idx][l_idx])
            constr_obj_2 *= c[b_idx] 

            for l_idx_2 in range(1, n_l+1):
                if l_idx != l_idx_2:
                    if u_b[b_idx][l_idx-1] < u_b[b_idx][l_idx_2-1]:
                        constr_obj_3 += (q[l_idx] * x[b_idx][l_idx])
                        
            model.addConstr((constr_obj_1 + constr_obj_2 + constr_obj_3) >= (c[b_idx]* (1-w[b_idx][l_idx])))
    
    u = {}
    for b_idx in range(1, n_b+1):
        u[b_idx] = {}
        for l_idx in range(1, n_l+1):
            u[b_idx][l_idx] = u_b[b_idx][l_idx-1] + u_l[l_idx][b_idx-1]
    
    model.setObjective(lambda_1*quicksum(u[b_idx][l_idx]*x[b_idx][l_idx] for l_idx in range(1, n_l+1) for b_idx in range(1, n_b+1)) - \
                       lambda_2*quicksum(w[b_idx][l_idx] for l_idx in range(1, n_l+1) for b_idx in range(1, n_b+1)) , GRB.MAXIMIZE)
    model.optimize()
    if model.status != 2:
        print("Optimal Solution not found !!!")
        return -1, -1
    
    borrower_matches = {}
    lender_matches = {}
    for b_idx in range(1, n_b+1):
        borrower_matches[b_idx] = []
#         print("Borrower {} matched to lenders: ".format(b_idx))
        for l_idx in range(1, n_l+1):
            if x[b_idx][l_idx].X == 1:
#                 print(l_idx)
                borrower_matches.append(l_idx)
                if l_idx not in lender_matches:
                    lender_matches[l_idx] = []
                lender_matches[l_idx].append(b_idx)
    
    return borrower_matches, lender_matches
#     items_selected = [i for i in range(n) if x[i].X > 0.5]
#     total_profit = int(model.ObjVal)
#     return items_selected, total_profit

In [20]:
def lender_utility(l, b, sim_values, amount_lenders, borrower_rates):
    return sim_values[b][l] + borrower_rates[b]*amount_lenders[l]

# Risk preference not considered for now
def borrower_utility(l, b, sim_values, risk_preference):
    return sim_values[b][l] #+ risk_preference[b][l]

In [25]:
from collections import defaultdict 

def rewards(mean, variance):
    return np.random.normal(mean, variance, 1000)

def average_reward(base_reward, reward_list):
    return base_reward + (rewards(1, 0.25) + np.sum(rewards_list))/(len(reward_list)+1) 

def reward_ucb(base_reward, reward_list, time):
    avg_reward = average_reward(u_l[l_idx][b_match-1], rewards_list_l[l_idx][b_match])
    return avg_reward + np.sqrt(2*np.log(time)/(len(reward_list)-1))
    

def gs_bandit_method_basic(n_b, n_l, u_b, u_l, lambda_1, lambda_2, preference_borrowers, preference_lenders):
    T = 100 # horizon, n
    
    curr_util_l = defaultdict(defaultdict(list))
    rewards_list_l = defaultdict(defaultdict(list))
    util_send_l = defaultdict(list)
    for l_idx in range(1, n_l+1):
        for b_idx in range(1, n_b+1):
            curr_util_l[l_idx][b_idx] = average_reward(u_l[l_idx][b_idx-1], rewards_list_l[l_idx][b_idx])
            util_send_l[l_idx].append(curr_util_l[l_idx][b_idx])
        
    for t in range(T):
        print("Matching time step " + str(t))
        borrower_matches, lender_matches = model_gs_matching(n_b, n_l, u_b, util_send_l, c, q, lambda_1, lambda_2, LogToConsole=False)
        
        for l_idx in range(1, n_l+1):
            b_match = lender_matches[l_idx] 
            rewards_list_l[l_idx][b_match].append(rewards(u_b[b_match][l_idx-1], 0.25))
            curr_util_l[l_idx][b_match] = average_reward(u_l[l_idx][b_match-1], rewards_list_l[l_idx][b_match])
            util_send_l[l_idx][b_match-1] = reward_ucb(u_l[l_idx][b_match-1], rewards_list_l[l_idx][b_match], t)
            
        
            print("Lender {}, matched to borrower {}".format(l_idx, b_idx))
    

In [23]:
def gs_bandit_method(n_b, n_l, preference_borrowers, preference_lenders):
    T = 100 # horizon, n
    
    for t in range(T):
        l_unmatched = n_l
        
        while len(l_unmatched) > 0:
            borrower_matches, lender_matches = model_gs_matching(n_b, n_l, u_b, u_l, c, q, lambda_1, lambda_2, LogToConsole=False)

            if borrower_matches == -1:
                print("Matching has reached an unstable configuration. Aborting!!!")
                break

            for l_idx in lender_matches:
                b_matched_l = lender_matches[l_idx][0] # every lender has 1 borrower
                b_most_preferred_l = preference_borrowers[b_idx][0]

                if b_matched_l == b_most_preferred_l:
                    

            
        
    return ""

In [ ]:
def simulate_lending():
    u_b = {}
    u_l = {}
    
    n_b, n_l = 3, 6

    preference_borrowers = []
    preference_lenders = []
    
    sim_values = {}
    for b_idx in range(1, n_b+1):
        sim_values[b_idx] = {}
        for l_idx in range(1, n_l+1):
            sim_values[b_idx][l_idx]  = random.uniform(0, 1)
    
    borrower_rates = {}
    for b_idx in range(1, n_b+1):
        borrower_rates[b_idx]  = random.uniform(0, 1)

    # This part is not used for now - risk_preference
    risk_preference = {}
    categories = list(range(1, 6))
    lender_risk_categories = {}
    for l_idx in range(1, l_idx+1):
        lender_risk_categories[l_idx] = {}
        for c_idx in categories:
            lender_risk_categories[l_idx][c_idx] = random.uniform(0, 1)
    borrower_categories = {}
    for b_idx in range(1, n_b+1):
        borrower_categories[b_idx] = random.sample(categories, 1)[0] # consider every borrower has 1 category for now
        risk_preference[b_idx] = {}
        for l_idx in range(1, n_l+1):
            risk_preference[b_idx][l_idx] = lender_risk_categories[l_idx][borrower_categories[b_idx]]
    
    # c - borrower amount, q - lender amount
    c = {}
    q = {}

    while True:
        sum_c = 0
        sum_q = 0
        for b_idx in range(1, n_b+1):
            c[b_idx] = random.sample(range(5, 10), 1)[0]
            sum_c += c[b_idx]

        for l_idx in range(1, n_l+1):
            q[l_idx] = random.sample(range(1, 10), 1)[0]
            sum_q += q[l_idx]

        if sum_q > sum_c:
            break

    # utilities
    for b_idx in range(1, n_b+1):
        u_b[b_idx] = []
        for l_idx in range(1, n_l+1):
            u_b[b_idx].append(borrower_utility(l_idx, b_idx, sim_values, risk_preference))
        preference_borrowers = sorted(range(len(u_b[b_idx])), key=lambda k: u_b[b_idx][k])

    for l_idx in range(1, n_l+1):
        u_l[l_idx] = []
        for b_idx in range(1, n_b+1):
            u_l[l_idx].append(lender_utility(l_idx, b_idx, sim_values, q, borrower_rates))
        preference_lenders = sorted(range(len(u_l[l_idx])), key=lambda k: u_l[l_idx][k])
    
    lambda_1 = 0.5
    lambda_2 = 1-lambda_1

    print("Configuration:")
    print("Borrower preferences: ", u_b)
    print("Borrower requests: ", c)
    print("Lender prefernces: ", u_l)
    print("Lender budgets: ", q)
    
    gs_bandit_method_basic(n_b, n_l, u_b, u_l, lambda_1, lambda_2, preference_borrowers, preference_lenders)
#     model_gs_matching(n_b, n_l, u_b, u_l, c, q, lambda_1, lambda_2, LogToConsole=False)

In [22]:
n_b = 3
n_l = 6

simulate_lending()

Configuration:
Borrower preferences:  {1: [0.5246363845197732, 0.664101277615801, 0.8911995046446982, 0.6449891450128602, 0.8417530125038343, 0.22088544997017234], 2: [0.4836304426425677, 0.2046860434026625, 0.24909375194167693, 0.480533532315052, 0.7593814478381152, 0.1629013828024004], 3: [0.7660974988752625, 0.6004812218353827, 0.10979712866762392, 0.5218979000158738, 0.08675975458049101, 0.21950888961934878]}
Borrower requests:  {1: 8, 2: 9, 3: 9}
Lender prefernces:  {1: [0.6034485881711393, 1.343092969856399, 2.424409711926506], 2: [0.9399439903955823, 3.2128048886510716, 6.404573967514734], 3: [1.2458544210758458, 4.116675124403917, 7.572202087398218], 4: [0.9996440614440076, 4.348114904777293, 7.984302858746468], 5: [0.9599713179808834, 2.048575238658862, 2.5742280741573564], 6: [0.4179159590985876, 2.3115577008369783, 4.365289422247457]}
Lender budgets:  {1: 2, 2: 7, 3: 9, 4: 9, 5: 3, 6: 5}
Borrower 1 matched to lenders: 
5
6
Borrower 2 matched to lenders: 
3
Borrower 3 matched